This notebook serves as a Gradio app functionality emulating GPT-4, and includes a LangChain agent for performing desired internet searches. The application relies on SerpAPI, a powerful tool that simplifies the process of executing Google searches through LangChain. I have utilized my GitHub account to create a SerpAPI account, which offers up to 100 free monthly internet searches. For additional details, please refer to the accompanying article.

- [Complete Guide: Connect OpenAI and Google Search using LangChain](https://medium.com/codingthesmartway-com-blog/complete-guide-connect-openai-and-google-search-using-langchain-43bbfb0dd264)

In [1]:
# Importing necessary modules
import os
from dotenv import load_dotenv
import openai
import gradio as gr
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.llms import OpenAI

/Users/markeyser/miniforge3/envs/transformers_nlp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables from the .env file
load_dotenv(".env")

# Retrieving API keys from the environment
openai.api_key = os.getenv("OPENAI_API_KEY")
serpapi_key = os.getenv("SERPAPI_API_KEY")

In [3]:
# Initialize the message_history global variable
message_history = [
    {"role": "user", "content": "You are a kind helpful assistant"},
    {"role": "assistant", "content": "OK"},
]

In [4]:
def clear_history():
    global message_history
    message_history = [
        {"role": "user", "content": "You are a kind helpful assistant"},
        {"role": "assistant", "content": "OK"},
    ]

In [6]:
def predict_with_search(input):
    return predict(input, search=True)

In [7]:
def predict(input, search=False):
    input_anonymized = input
    message_history.append({"role": "user", "content": input_anonymized})

    if search:
        llm = OpenAI(temperature=0)
        tools = load_tools(["serpapi", "llm-math"], llm=llm)
        agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
        reply_content = agent.run(input_anonymized)
    else:
        completion = openai.ChatCompletion.create(model="gpt-4", messages=message_history)
        reply_content = completion.choices[0].message.content

    message_history.append({"role": "assistant", "content": reply_content})

    response = [
        (message_history[i]["content"], message_history[i + 1]["content"])
        for i in range(2, len(message_history) - 1, 2)
    ]
    return response

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## Welcome!")

    chatbot = gr.Chatbot()

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)
        search_internet = gr.Button("Search Internet")
        clear = gr.Button("New chat")

    txt.submit(predict, txt, chatbot)
    txt.submit(None, None, txt, _js="() => {''}")
    search_internet.click(predict_with_search, txt, chatbot)
    clear.click(clear_history, None, chatbot, queue=False)

In [8]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://16bb8c099ca07c994d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
 I should search for information about this library.
Action: Search
Action Input: "LangChain Python library"
Observation: This library aims to assist in the development of those types of applications. Common examples of these applications include:.
Thought: I should read more about the library.
Action: Search
Action Input: "LangChain Python library usage"
Observation: Langchain library is an innovative Python framework designed to enhance the efficiency of working with extensive language models. It provides developers with a comprehensive set of abstractions, implementations, and components for building applications that leverage the power of language models.
Thought: I now know the final answer.
Final Answer: LangChain Python library is an innovative Python framework designed to enhance the efficiency of working with extensive language models. It provides developers with a comprehensive set of abstractions, implementations, and components for bu